# Notebook1 Prepare Dataset

Readme: In this notebook, CO2 Emission data is preprocessed, and two types of dataset are saved for the following two work streams.

- **Content in this notebook:** Data Processing

    - Feature Engineering: Divide two Features: `Transmission` and `Model`

    - Data Encoding: Change all the categorical variable into dummy variable

    - Dataset Division: Split the two dataset into train and test dataset (set random seeed to maintain reprocducibility)

    - Dataset Saving: Save the four files: dataset1-train, dataset1-test, dataset2-train, dataset2-test.

- **Input/Output of this notebook:**

     - Input: *CO2 Emissions_Canada.csv*

     - Output: four csv files: 
     
        *Dataset1_train.csv*, *Dataset1_test.csv*,
        
        *Dataset2_train.csv*, *Dataset2_test.csv*

Note: In this part, data standerdization is not included. 

## Import Modules

In [465]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

## Import Data

In [466]:
df_co2 = pd.read_csv('CO2 Emissions_Canada.csv')
df_co2.head(3)
df_co2.shape # (7385, 12)

(7385, 12)

## Feature Manipulation

### Drop Nature Gas Types

We decide that Nature Gas Type is not included in the following analysis.

In [467]:
df_co2 = df_co2[df_co2["Fuel Type"] != "N"]
df_co2.shape # (7385, 12)

(7384, 12)

### Transmission Feature Split

In [468]:
def gearbox_type(data):
    Automatic = "A"
    Manual = "M"
    encoded = 9999
    if Automatic in data:
        encoded = 1
    if Manual in data:
        encoded = 0
    return encoded

def gear_number(data):
    encoded = 9999
    number = list(data)[-1]
    no_change_needed = list("456789")
    if number in no_change_needed:
        return float(number)
    elif number in "0":
        return 10.0
    elif number in "V":
        return float(1926/281)
    else:
        return encoded

In [469]:
df_co2["Gearbox_Type"] = df_co2["Transmission"].map(gearbox_type)
df_co2["Gearbox_Type"].value_counts()

Gearbox_Type
1    5553
0    1831
Name: count, dtype: int64

In [470]:
df_co2["Gearbox_Number"] = df_co2["Transmission"].map(gear_number)
df_co2["Gearbox_Number"].value_counts()

Gearbox_Number
6.000000     3258
8.000000     1802
7.000000     1026
9.000000      419
5.000000      307
6.854093      295
10.000000     210
4.000000       67
Name: count, dtype: int64

### Vehicle Model Features

In [471]:
def vehicle_feature(data):
    data_string = data.split(" ")
    for item in data_string:
        code = None
        if "4WD" in item:
            code = "Four-wheel drive"
        if "4X4" in item:
            code = "Four-wheel drive"
        if "FFV" in item:
            code = "Flexible-fuel vehicle"
        if "SWB" in item:
            code = "Short wheelbase"
        if "LWB" in item:
            code = "Long wheelbase"
        if "EWB" in item:
            code = "Extended wheelbase"
    return code

In [472]:
df_co2["Model Features"] = df_co2["Model"].map(vehicle_feature)
df_co2["Model Features"].value_counts()

Model Features
Four-wheel drive         549
Flexible-fuel vehicle    496
Long wheelbase            26
Short wheelbase           15
Extended wheelbase        15
Name: count, dtype: int64

#### Add Fuel 

In [473]:
def get_KM_per_MJ(fuel_type,fuel_econ):
    # All in MJ per L results in MJ/100km
    energy_density = {"X": 34.2, "Z": 34.2, "E": 24.0, "D": 38.6}
    value = 0
    if fuel_type == "X" or fuel_type == "Z":
        return fuel_econ * energy_density["X"]
    if fuel_type == "D":
        return fuel_econ * energy_density["D"]
    if fuel_type == "E":
        return fuel_econ * energy_density["E"]
    return value



#### Add Energy Per Distance

In [474]:
df_co2["Energy Consumption per Distance MJ/100km"] = df_co2[["Fuel Type","Fuel Consumption Comb (L/100 km)"]].apply(lambda x: get_KM_per_MJ(*x), axis=1)


In [475]:
df_co2

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km),Gearbox_Type,Gearbox_Number,Model Features,Energy Consumption per Distance MJ/100km
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196,1,5.0,None,290.70
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221,0,6.0,None,328.32
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136,1,7.0,None,201.78
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255,1,6.0,Four-wheel drive,379.62
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244,1,6.0,None,362.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,VOLVO,XC40 T5 AWD,SUV - SMALL,2.0,4,AS8,Z,10.7,7.7,9.4,30,219,1,8.0,None,321.48
7381,VOLVO,XC60 T5 AWD,SUV - SMALL,2.0,4,AS8,Z,11.2,8.3,9.9,29,232,1,8.0,None,338.58
7382,VOLVO,XC60 T6 AWD,SUV - SMALL,2.0,4,AS8,Z,11.7,8.6,10.3,27,240,1,8.0,None,352.26
7383,VOLVO,XC90 T5 AWD,SUV - STANDARD,2.0,4,AS8,Z,11.2,8.3,9.9,29,232,1,8.0,None,338.58


## Set Dataset for following workflow

For creating dataset for the following two streams:

- Delete `Model`, replaced by  `Model Features`

- Delete `Transmission`, replaced by `Gearbox_Type` and `Gearbox_Number`

- **Dataset1**: Used for Current Vehicle Manufacturer Benchmarking, 

- **Dataset2**: Used for Future Vehicle Design Evaluation, which means fuel efficiency related is not available, thus **drop** columns `Fuel Consumption Hwy (L/100 km)`, `Fuel Consumption Comb (L/100 km)`, and `Fuel Consumption Comb (mpg)`.

In [476]:
# For workstream 1, current state benchmarking
dataset1 = df_co2[['Model Features', 'Vehicle Class', 'Engine Size(L)', 'Cylinders',
       'Gearbox_Type', 'Gearbox_Number', 'Fuel Type', 'Fuel Consumption City (L/100 km)',
       'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)',
       'Fuel Consumption Comb (mpg)', 'CO2 Emissions(g/km)']]  

# For workstream 2, future design evaluation
dataset2 = df_co2[['Model Features', 'Vehicle Class', 'Engine Size(L)', 'Cylinders',
       'Gearbox_Type', 'Gearbox_Number', 'Fuel Type', 'CO2 Emissions(g/km)', "Energy Consumption per Distance MJ/100km"]]



In [477]:
dataset2.describe()

,Engine Size(L),Cylinders,Gearbox_Type,Gearbox_Number,CO2 Emissions(g/km),Energy Consumption per Distance MJ/100km
count,7384.000000,7384.000000,7384.000000,7384.000000,7384.000000,7384.000000
mean,3.160008,5.614978,0.752031,6.885422,250.589789,367.642874
std,1.354252,1.828425,0.431863,1.184940,58.515007,86.067362
min,0.900000,3.000000,0.000000,4.000000,96.000000,140.220000
25%,2.000000,4.000000,1.000000,6.000000,208.000000,304.380000
50%,3.000000,6.000000,1.000000,6.854093,246.000000,359.100000
75%,3.700000,6.000000,1.000000,8.000000,288.000000,422.400000
max,8.400000,16.000000,1.000000,10.000000,522.000000,759.240000


Set random seed to guarantee reproducibility

In [478]:
import random

# Set Random seeds
random.seed(42)
np.random.seed(42)

Encode categorical features

In [479]:
cat_features_set1 = dataset1.select_dtypes(include=['object']).columns  # select categorical columns
cat_features_set2 = dataset2.select_dtypes(include=['object']).columns  # select categorical columns

dataset1_encoded = pd.get_dummies(dataset1, columns=cat_features_set1, drop_first=True, dtype=int)
dataset2_encoded = pd.get_dummies(dataset2, columns=cat_features_set2, drop_first=True, dtype=int)

dataset1_encoded['Make'] = df_co2['Make']
dataset2_encoded['Make'] = df_co2['Make']

Divide Train Dataset and Test Dataset

In [480]:
from sklearn.model_selection import train_test_split

# Prepare for Dataset 1
dataset1_train_features = dataset1_encoded.drop(columns=['CO2 Emissions(g/km)'])
dataset1_train_y = dataset1_encoded['CO2 Emissions(g/km)']

dataset1_features_train, dataset1_features_test, dataset1_y_train, dataset1_y_test = train_test_split(
    dataset1_train_features, 
    dataset1_train_y, 
    test_size=0.2, random_state=42)

# Combine X & y into one data set
Dataset1_train = pd.concat([dataset1_features_train, dataset1_y_train], axis=1)
Dataset1_test = pd.concat([dataset1_features_test, dataset1_y_test], axis=1)

Dataset1_test.head()

,Engine Size(L),Cylinders,Gearbox_Type,Gearbox_Number,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),Model Features_Flexible-fuel vehicle,Model Features_Four-wheel drive,...,Vehicle Class_SUV - SMALL,Vehicle Class_SUV - STANDARD,Vehicle Class_TWO-SEATER,Vehicle Class_VAN - CARGO,Vehicle Class_VAN - PASSENGER,Fuel Type_E,Fuel Type_X,Fuel Type_Z,Make,CO2 Emissions(g/km)
5632,6.2,8,0,7.0,18.2,12.5,15.6,18,0,0,...,0,0,1,0,0,0,0,1,CHEVROLET,368
1550,3.6,6,1,6.0,14.8,9.9,12.6,22,0,0,...,1,0,0,0,0,0,1,0,GMC,290
1128,4.2,8,0,6.0,20.5,11.7,16.6,17,0,0,...,0,0,1,0,0,0,0,1,AUDI,382
6498,2.0,4,1,8.0,10.3,7.5,9.0,31,0,0,...,1,0,0,0,0,0,0,1,VOLVO,211
3270,1.8,4,1,6.0,9.4,6.8,8.2,34,0,0,...,0,0,0,0,0,0,1,0,VOLKSWAGEN,193


In [481]:

# Prepare for Dataset 2
dataset2_train_features = dataset2_encoded.drop(columns=['CO2 Emissions(g/km)', 'Make'])
dataset2_train_y = dataset2_encoded['CO2 Emissions(g/km)']

dataset2_features_train, dataset2_features_test, dataset2_y_train, dataset2_y_test = train_test_split(
    dataset2_train_features, 
    dataset2_train_y, 
    test_size=0.2, random_state=42)

# Combine X & y into one data set
Dataset2_train = pd.concat([dataset2_features_train, dataset2_y_train, dataset2_encoded['Make']], axis=1)
Dataset2_test = pd.concat([dataset2_features_test, dataset2_y_test, dataset2_encoded['Make']], axis=1)

Dataset2_test.head()

,Engine Size(L),Cylinders,Gearbox_Type,Gearbox_Number,Energy Consumption per Distance MJ/100km,Model Features_Flexible-fuel vehicle,Model Features_Four-wheel drive,Model Features_Long wheelbase,Model Features_Short wheelbase,Vehicle Class_FULL-SIZE,...,Vehicle Class_SUV - SMALL,Vehicle Class_SUV - STANDARD,Vehicle Class_TWO-SEATER,Vehicle Class_VAN - CARGO,Vehicle Class_VAN - PASSENGER,Fuel Type_E,Fuel Type_X,Fuel Type_Z,CO2 Emissions(g/km),Make
5632,6.2,8.0,0.0,7.0,533.52,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,368.0,CHEVROLET
1550,3.6,6.0,1.0,6.0,430.92,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,290.0,GMC
1128,4.2,8.0,0.0,6.0,567.72,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,382.0,AUDI
6498,2.0,4.0,1.0,8.0,307.80,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,211.0,VOLVO
3270,1.8,4.0,1.0,6.0,280.44,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,193.0,VOLKSWAGEN


Save Output

In [482]:
Dataset1_train.to_csv('Dataset1_train.csv', index=False)
Dataset1_test.to_csv('Dataset1_test.csv', index=False)

Dataset2_train.to_csv('Dataset2_train.csv', index=False)
Dataset2_test.to_csv('Dataset2_test.csv', index=False)

In [483]:
Dataset1_train

,Engine Size(L),Cylinders,Gearbox_Type,Gearbox_Number,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),Model Features_Flexible-fuel vehicle,Model Features_Four-wheel drive,...,Vehicle Class_SUV - SMALL,Vehicle Class_SUV - STANDARD,Vehicle Class_TWO-SEATER,Vehicle Class_VAN - CARGO,Vehicle Class_VAN - PASSENGER,Fuel Type_E,Fuel Type_X,Fuel Type_Z,Make,CO2 Emissions(g/km)
6274,4.0,6,1,5.000000,14.7,10.3,12.7,22,0,0,...,0,0,0,0,0,0,1,0,NISSAN,299
1608,2.4,4,1,6.000000,9.8,6.7,8.4,34,0,0,...,0,0,0,0,0,0,1,0,HYUNDAI,193
2291,2.0,4,1,8.000000,11.3,7.1,9.4,30,0,0,...,0,0,0,0,0,0,0,1,BMW,221
3232,2.5,4,1,6.000000,10.1,7.7,9.0,31,0,0,...,1,0,0,0,0,0,1,0,TOYOTA,212
3881,5.0,8,1,8.000000,15.8,10.3,13.3,21,0,0,...,0,0,0,0,0,0,0,1,JAGUAR,313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,2.0,4,1,6.000000,9.3,7.3,8.4,34,0,0,...,0,0,0,0,0,0,0,1,MINI,197
5227,5.6,8,1,7.000000,17.5,12.8,15.4,18,0,1,...,0,1,0,0,0,0,1,0,NISSAN,362
5391,3.5,6,1,6.000000,8.1,8.5,8.3,34,0,0,...,0,1,0,0,0,0,1,0,TOYOTA,193
860,2.5,4,1,6.854093,9.5,7.4,8.6,33,0,0,...,1,0,0,0,0,0,1,0,NISSAN,198
